In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
conf = SparkConf().setAppName("ejemplo1")
sc = SparkContext(conf = conf)

In [3]:
df1 = sc.range(2,10000000,2)
df2 = sc.range(2,10000000,4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step1 = step1.select("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect()

AttributeError: 'RDD' object has no attribute 'select'

In [4]:
df1.take(5)

[2, 4, 6, 8, 10]

In [5]:
sc

<SparkContext master=local[*] appName=ejemplo1>

In [6]:
sc.stop()

In [7]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Ejemplo1_SparkSession") \
    .getOrCreate()

In [8]:
spark

In [9]:
df1 = spark.range(2,10000000,2)
df2 = spark.range(2,10000000,4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect()

[Row(sum(id)=2500000000000)]

In [19]:
type(df1)

pyspark.sql.dataframe.DataFrame

In [16]:
step4.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[sum(id#8L)])
+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#66]
   +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#8L)])
      +- *(6) Project [id#8L]
         +- *(6) SortMergeJoin [id#8L], [id#2L], Inner
            :- *(3) Sort [id#8L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [id=#50]
            :     +- *(2) Project [(id#0L * 5) AS id#8L]
            :        +- Exchange RoundRobinPartitioning(5), REPARTITION_WITH_NUM, [id=#46]
            :           +- *(1) Range (2, 10000000, step=2, splits=1)
            +- *(5) Sort [id#2L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#2L, 200), ENSURE_REQUIREMENTS, [id=#57]
                  +- Exchange RoundRobinPartitioning(6), REPARTITION_WITH_NUM, [id=#56]
                     +- *(4) Range (2, 10000000, step=4, splits=1)


